<a href="https://colab.research.google.com/github/naafey-aamer/Malaria_be_gone/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [ ]:
#from tensorflow.org
(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'malaria',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,)

In [ ]:
IMG_SIZE = 120

#from tensorflow.org
def format_example(image, label):
  image = tf.cast(image, tf.float32)
  image = tf.image.random_flip_left_right(image)  # Model overfits after 3-4 EPOCHS so Randomized Data augmentation applied
  image = image / 255
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image, label


train = raw_train.map(format_example)
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)


In [ ]:
print(len(train))
print(len(validation))
print(len(test))

22046
2756
2756


In [ ]:
BATCH_SIZE = 256 #Lower sized batches are overfitting
SHUFFLE_SIZE = 1000
train_ds = train.shuffle(SHUFFLE_SIZE).batch(BATCH_SIZE)
validation_ds = validation.batch(BATCH_SIZE)
test_ds = test.batch(BATCH_SIZE)

In [ ]:
#settled on this shape keeping computational costs in mind and preventing overfitting
#average pooling not working well
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
  tf.keras.layers.MaxPooling2D(2, 2),

  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),

  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),

  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),

  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.5), #Dropout significantly preventing overfitting
  tf.keras.layers.Dense(2, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 118, 118, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 59, 59, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 57, 57, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 28, 28, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 26, 26, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 128)      0

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping #early stopping to prevent overfitting

def lr_schedule(epoch, lr):
    # Reduce learning rate by 20% every 2 epochs
    if (epoch+1) % 2 == 0:
        lr *= 0.8
    return max(lr, 3e-4)

EPOCHS = 12

lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=2)

model.fit(train_ds,
          validation_data=validation_ds,
          epochs=EPOCHS,
          callbacks=[early_stop, lr_callback])



Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/12
87/87 [==============================] - 865s 10s/step - loss: 0.4828 - accuracy: 0.7446 - val_loss: 0.1956 - val_accuracy: 0.9332 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.000800000037997961.
Epoch 2/12
87/87 [==============================] - 847s 10s/step - loss: 0.1682 - accuracy: 0.9469 - val_loss: 0.1587 - val_accuracy: 0.9478 - lr: 8.0000e-04

Epoch 3: LearningRateScheduler setting learning rate to 0.000800000037997961.
Epoch 3/12
87/87 [==============================] - 850s 10s/step - loss: 0.1456 - accuracy: 0.9536 - val_loss: 0.1471 - val_accuracy: 0.9496 - lr: 8.0000e-04

Epoch 4: LearningRateScheduler setting learning rate to 0.0006400000303983689.
Epoch 4/12
87/87 [==============================] - 843s 10s/step - loss: 0.1382 - accuracy: 0.9550 - val_loss: 0.1401 - val_accuracy: 0.9517 - lr: 6.4000e-04

Epoch 5: LearningRateScheduler setting learning r

In [ ]:
loss, accuracy = model.evaluate(test_ds)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

11/11 [==============================] - 30s 3s/step - loss: 0.1406 - accuracy: 0.9583
Loss: 0.14063197374343872
Accuracy: 0.9582728743553162
